# Snowtornado

In [119]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from stock_exchange import create_interval_tree

In [200]:
products = pd.read_csv("data/cellphone.csv", dtype=str, index_col="id")
matches = pd.read_csv("data/matches_cellphone.csv", index_col=False, usecols=[0, 1, 2], names=['original_id', 'replacement_id', 'timestamp'])
matches['timestamp'] = pd.to_datetime(matches['timestamp'])
stock_exchange = create_interval_tree("data/stock_exchange_cellphone.csv")

Clean data by converting every field to lowercase, stripping, and finally converting it to a category type

In [155]:
binary = pd.DataFrame()

for column in products.columns:
    if not column in ["name"]:
        binary[column] = products[column].str.lower().str.strip().astype('category')
    
binary = pd.get_dummies(binary, dummy_na=True)
binary = binary.loc[:, (binary != 0).any(axis=0)] # Remove superfluous NaN columns
#for column in binary.columns:
#    print(column)
binary.shape

(1876, 738)

In [156]:
matches_train, matches_test = train_test_split(matches, test_size=0.3, random_state=42)

Fill training data with the binary data

In [157]:
X_train = np.zeros((matches_train.shape[0], binary.shape[1]), dtype = 'bool')
X_test = np.zeros((matches_test.shape[0], binary.shape[1]), dtype = 'bool')
Y_train = np.zeros((matches_train.shape[0], binary.shape[1]), dtype = 'bool')
Y_test = np.zeros((matches_test.shape[0], binary.shape[1]), dtype = 'bool')

In [158]:
i = 0
for _, row in matches_train.iterrows():
    original_id = row['original_id']
    replacement_id = row['replacement_id']
    X_train[i] = binary.loc[[original_id]].values[0]
    Y_train[i] = binary.loc[[replacement_id]].values[0]
    i += 1

i = 0
for _, row in matches_test.iterrows():
    original_id = row['original_id']
    replacement_id = row['replacement_id']
    X_test[i] = binary.loc[[original_id]].values[0]
    Y_test[i] = binary.loc[[replacement_id]].values[0]
    i += 1

In [203]:
# The following model could still be supplemanted by something like a tf-idf preprocess. 
model = DecisionTreeRegressor(max_depth=6).fit(X_train, Y_train)
model.score(X_test, Y_test)

0.68244177112864124

In [208]:
# Maybe the scoring can be improved upon? What we want is not R^2 but the correct 'directions'
# If cosine similarity is what is going to be used then that could be used to determine the 
# performance of the algorithm?
#np.square(cosine_similarity(model.predict(X_test), Y_test) - 1).sum() / X_test.shape[0]
cosine_similarity(model.predict(X_test), Y_test).mean()

0.73263570096879671

In [212]:
def products_on_the_market(timestamp):
    return binary.loc[set(map((lambda x: x.data), stock_exchange[timestamp]))]

In [215]:
def find_matches(original_id, timestamp, limit = 3):
    prediction = model.predict(binary.loc[[original_id]].values)
    similar = cosine_similarity(prediction, products_on_the_market(timestamp))[0]
    indices = np.argpartition(similar, -limit)
    indices = indices[np.argsort(similar[indices])][::-1]
    return products_on_the_market(timestamp).index.values[indices]

In [216]:
from IPython.display import display, HTML
pd.options.display.max_columns = 100
from itertools import islice
from datetime import datetime

for _, row in islice(matches_test.iterrows(), 10):
    original_id = row['original_id']
    replacement_id = row['replacement_id']
    timestamp = row['timestamp']
    
    # Dates in early 2014 shows abnormally few products on the market
    if timestamp < datetime(2014, 6, 1, 12, 0, 0):
        next
    else:
        display(products.loc[[original_id, replacement_id, find_matches(original_id, timestamp, 1)[0]]])

,name,producer,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
76427,Xperia Z3,Sony Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,5248x3936,4096x2160,5.2,16000000,NaN,NaN,Standard,LCD,1920x1080,16,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,21,NaN,4,Krait,NaN,2500,Nej,Ja,Snapdragon 801,3072,Nej,Ja,Ja,FULL HD,Nej
76427,Xperia Z3,Sony Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,5248x3936,4096x2160,5.2,16000000,NaN,NaN,Standard,LCD,1920x1080,16,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,21,NaN,4,Krait,NaN,2500,Nej,Ja,Snapdragon 801,3072,Nej,Ja,Ja,FULL HD,Nej
76427,Xperia Z3,Sony Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,5248x3936,4096x2160,5.2,16000000,NaN,NaN,Standard,LCD,1920x1080,16,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,21,NaN,4,Krait,NaN,2500,Nej,Ja,Snapdragon 801,3072,Nej,Ja,Ja,FULL HD,Nej


,name,producer,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
48478,One S,HTC,NaN,Quad-Band,Ja,Integrerad,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,Ja,Ja,NaN,Ja,NaN,Ja,Ja,NaN,Ja,Ja,Polyfoniska. MP3,Ja,Ja,Ja,Nej,3264x2448,1920x1080,4.3,16000000,NaN,NaN,Standard,OLED,960x540,16,NaN,Ja,Ja,Ja,Android,Ja,Ja,Nej,8,Nej,2,Krait,Nej,1500,Nej,Nej,Snapdragon S4,1024,Nej,Nej,Nej,< HD,Nej
64481,One Mini,HTC,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,Ja,Ja,Ja,NaN,2688x1520,1920x1080,4.3,16000000,NaN,NaN,Standard,OLED,1280x720,16,NaN,NaN,Ja,Ja,Android,Ja,NaN,Nej,4,Nej,2,Krait,NaN,1400,Nej,Ja,Snapdragon S4,1024,Nej,Nej,Nej,HD,Nej
61947,Galaxy S4 Mini i9195 8GB,Samsung Electronics Svenska AB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,Ja,Ja,Ja,NaN,3264x2448,1920x1080,4.3,16000000,NaN,NaN,Standard,OLED,960x540,8,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,8,Nej,2,Krait,NaN,1700,Nej,Ja,Snapdragon S4,1536,Nej,Ja,Nej,< HD,Nej


,name,producer,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
66274,iPhone 5s 32GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,NaN,Ja,Ja,Nej,NaN,3264x2448,1920x1080,4,16000000,NaN,NaN,Standard,LCD,1136x640,32,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,Nej,2,NaN,NaN,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej
66274,iPhone 5s 32GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,NaN,Ja,Ja,Nej,NaN,3264x2448,1920x1080,4,16000000,NaN,NaN,Standard,LCD,1136x640,32,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,Nej,2,NaN,NaN,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej
66274,iPhone 5s 32GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,NaN,Ja,Ja,Nej,NaN,3264x2448,1920x1080,4,16000000,NaN,NaN,Standard,LCD,1136x640,32,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,Nej,2,NaN,NaN,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej


,name,producer,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
57291,Galaxy S4 i9505 16GB,Samsung Electronics Svenska AB,NaN,Quad-Band,Nej,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,NaN,NaN,Ja,NaN,Ja,Ja,Ja,Nej,4128x3096,1920x1080,5,16000000,NaN,NaN,Standard,OLED,1920x1080,16,NaN,Ja,Ja,Ja,Android,Ja,Ja,Ja,13,Nej,4,Krait,Ja,1900,Nej,Ja,Snapdragon 600,2048,Nej,Ja,Nej,FULL HD,Nej
67475,Galaxy S4 i9506 16GB,Samsung Electronics Svenska AB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,Ja,Ja,Ja,NaN,4000x3000,1920x1080,5,16000000,NaN,NaN,Standard,OLED,1920x1080,16,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,13,Nej,4,Krait,NaN,2300,Nej,Ja,Snapdragon 800,2048,Nej,Ja,Nej,FULL HD,Nej
67475,Galaxy S4 i9506 16GB,Samsung Electronics Svenska AB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,Ja,Ja,Ja,NaN,4000x3000,1920x1080,5,16000000,NaN,NaN,Standard,OLED,1920x1080,16,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,13,Nej,4,Krait,NaN,2300,Nej,Ja,Snapdragon 800,2048,Nej,Ja,Nej,FULL HD,Nej


,name,producer,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
66274,iPhone 5s 32GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,NaN,Ja,Ja,Nej,NaN,3264x2448,1920x1080,4,16000000,NaN,NaN,Standard,LCD,1136x640,32,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,Nej,2,NaN,NaN,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej
66274,iPhone 5s 32GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,NaN,Ja,Ja,Nej,NaN,3264x2448,1920x1080,4,16000000,NaN,NaN,Standard,LCD,1136x640,32,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,Nej,2,NaN,NaN,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej
66274,iPhone 5s 32GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,NaN,Ja,Ja,Nej,NaN,3264x2448,1920x1080,4,16000000,NaN,NaN,Standard,LCD,1136x640,32,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,Nej,2,NaN,NaN,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej


,name,producer,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1334x750,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,Cyclone,NaN,1400,Nej,Ja,A8,1024,Ja,Nej,Nej,HD,Nej
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1334x750,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,Cyclone,NaN,1400,Nej,Ja,A8,1024,Ja,Nej,Nej,HD,Nej
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1334x750,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,Cyclone,NaN,1400,Nej,Ja,A8,1024,Ja,Nej,Nej,HD,Nej


,name,producer,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
44287,iPhone 4s 16GB,Apple Sverige,NaN,Quad-Band,Ja,Integrerad,Ja,NaN,NaN,14,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,Ja,Ja,200,Ja,NaN,Ja,Ja,NaN,Ja,Nej,Polyfoniska. MP3,Ja,Ja,Ja,Nej,3264x2448,1920x1080,3.5,16000000,NaN,NaN,Standard,LCD,960x640,16,NaN,Ja,Ja,Ja,iOS,Ja,Ja,Nej,8,NaN,2,ARM Cortex-A9,NaN,800,Nej,Nej,A5,512,Nej,Nej,Nej,< HD,Nej
66273,iPhone 5s 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,NaN,Ja,Ja,Nej,NaN,3264x2448,1920x1080,4,16000000,NaN,NaN,Standard,LCD,1136x640,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,Nej,2,NaN,NaN,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej
66273,iPhone 5s 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,NaN,Ja,Ja,Nej,NaN,3264x2448,1920x1080,4,16000000,NaN,NaN,Standard,LCD,1136x640,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,Nej,2,NaN,NaN,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej


,name,producer,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1334x750,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,Cyclone,NaN,1400,Nej,Ja,A8,1024,Ja,Nej,Nej,HD,Nej
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1334x750,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,Cyclone,NaN,1400,Nej,Ja,A8,1024,Ja,Nej,Nej,HD,Nej
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1334x750,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,Cyclone,NaN,1400,Nej,Ja,A8,1024,Ja,Nej,Nej,HD,Nej


,name,producer,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1334x750,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,Cyclone,NaN,1400,Nej,Ja,A8,1024,Ja,Nej,Nej,HD,Nej
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1334x750,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,Cyclone,NaN,1400,Nej,Ja,A8,1024,Ja,Nej,Nej,HD,Nej
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1334x750,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,Cyclone,NaN,1400,Nej,Ja,A8,1024,Ja,Nej,Nej,HD,Nej


,name,producer,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1334x750,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,Cyclone,NaN,1400,Nej,Ja,A8,1024,Ja,Nej,Nej,HD,Nej
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1334x750,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,Cyclone,NaN,1400,Nej,Ja,A8,1024,Ja,Nej,Nej,HD,Nej
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1334x750,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,Cyclone,NaN,1400,Nej,Ja,A8,1024,Ja,Nej,Nej,HD,Nej
